In [3]:
import numpy as np
import tensorflow as tough
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from skimage.transform import resize
from tensorflow.keras.regularizers import l2

def loadingData():
    data_full = np.load('data_train.npy')
    labels_all = np.load('labels_train.npy')

    #Reshape and Resize data
    reshaped_data = data.T.reshape((-1, 300, 300, 3))
    resizing_data = np.array([resize(img, (100, 100, 3), anti_aliasing=True) for img in reshaped_data])

    # Preprocessing the data
    normalizingData = resizing_data / 255.0

    return normalizingData, labels_all

def building_model():
    model = Sequential([
        Conv2D(64, (3, 3), activation='relu', input_shape=(100, 100, 3)),
        BatchNormalization(),
        MaxPooling2D((2, 2)),

        Conv2D(128, (3, 3), activation='relu'),
        BatchNormalization(),
        MaxPooling2D((2, 2)),

        Conv2D(256, (3, 3), activation='relu'),
        BatchNormalization(),
        MaxPooling2D((2, 2)),

        Conv2D(512, (3, 3), activation='relu'),
        BatchNormalization(),
        MaxPooling2D((2, 2)),

        #Increasing Complexity of the model and adjusting dropout rate with more Dense
        Flatten(),
        Dense(1024, activation='relu', kernel_regularizer=l2(0.001)),
        Dropout(0.4),
        Dense(512, activation='relu', kernel_regularizer=l2(0.001)),
        Dropout(0.3),
        Dense(9, activation='softmax')
    ])

    return model

def trainingSet():
    data_full, labels_all = loadingData()

    # Using Data augmentation generator and increasing it.
    datagen = ImageDataGenerator(
        rotation_range=20,
        width_shift_range=0.15,
        height_shift_range=0.15,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest'
    )

    # Splitting the data into 80% of Training data and 20% Testing data.
    train_data, val_data, train_labels, val_labels = train_test_split(data_full, labels_all, test_size=0.2, random_state=42)

    
    model = building_model()
    model.compile(optimizer=Adam(learning_rate=0.0003), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    # Callbacks which keeps an eye on validation_loss:
    earlyStopping = tough.keras.callbacks.EarlyStopping(monitor='val_loss', patience=15)
    learningRate_scheduler = tough.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=7, min_lr=0.00001)
    modellingCheckpoint = tough.keras.callbacks.ModelCheckpoint('my_best_model.h5', save_best_only=True)

    # Training the model with the parameters given in the callbacks functions.
    trainingGenerator = datagen.flow(train_data, train_labels, batch_size=32)
    model.fit(trainingGenerator, epochs=60, validation_data=(val_data, val_labels),
              callbacks=[earlyStopping, learningRate_scheduler, modellingCheckpoint])


if __name__ == "__main__":
    trainingSet()


Epoch 1/60
212/212 [==============================] - 12s 52ms/step - loss: 5.2616 - accuracy: 0.1220 - val_loss: 6.0595 - val_accuracy: 0.1113 - lr: 3.0000e-04
Epoch 2/60
212/212 [==============================] - 11s 50ms/step - loss: 4.6421 - accuracy: 0.1334 - val_loss: 4.8237 - val_accuracy: 0.1113 - lr: 3.0000e-04
Epoch 3/60
212/212 [==============================] - 11s 50ms/step - loss: 4.3487 - accuracy: 0.1516 - val_loss: 4.1800 - val_accuracy: 0.1551 - lr: 3.0000e-04
Epoch 4/60
212/212 [==============================] - 11s 50ms/step - loss: 4.0602 - accuracy: 0.1697 - val_loss: 3.8687 - val_accuracy: 0.2362 - lr: 3.0000e-04
Epoch 5/60
212/212 [==============================] - 11s 50ms/step - loss: 3.7566 - accuracy: 0.2039 - val_loss: 3.7514 - val_accuracy: 0.1652 - lr: 3.0000e-04
Epoch 6/60
212/212 [==============================] - 11s 50ms/step - loss: 3.4658 - accuracy: 0.2431 - val_loss: 3.3578 - val_accuracy: 0.2410 - lr: 3.0000e-04
Epoch 7/60
212/212 [==============